In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

#### -> loading dataset

In [2]:
car_sales_missing = pd.read_csv("car-sales-extended-missing-data.csv")
car_sales_missing

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


#### -> removing empty spaces in y label

In [3]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [4]:
len(car_sales_missing)

1000

In [5]:
car_sales_missing.dropna(subset=["Price"], inplace = True)

In [6]:
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [7]:
# samples reduced
len(car_sales_missing)

950

#### -> splitting data into X and y labels

In [8]:
X = car_sales_missing.drop("Price", axis = 1)
y = car_sales_missing["Price"]

#### -> filling empty spaces in X labels

In [9]:
cat_imputer = SimpleImputer(strategy="constant", fill_value = "missing")
door_imputer = SimpleImputer(strategy = "constant", fill_value = 4)
num_imputer = SimpleImputer(strategy = "mean")

In [10]:
#defining columns
cat_features = ["Make", "Colour"]
door_feature = ["Doors"]
num_features = ["Odometer (KM)"]

In [11]:
# creating an imputer
imputer = ColumnTransformer([
            ("cat_imputer", cat_imputer, cat_features),
            ("num_imputer", num_imputer, num_features),
            ("door_imputer", door_imputer, door_feature)
])

In [12]:
# filling the empty spaces
filled_X = imputer.fit_transform(X)
filled_X

array([['Honda', 'White', 35431.0, 4.0],
       ['BMW', 'Blue', 192714.0, 5.0],
       ['Honda', 'White', 84714.0, 4.0],
       ...,
       ['Nissan', 'Blue', 66604.0, 4.0],
       ['Honda', 'White', 215883.0, 4.0],
       ['Toyota', 'Blue', 248360.0, 4.0]], dtype=object)

In [13]:
car_sales_filled = pd.DataFrame(filled_X, columns = ["Make", "Colour", "Odometer (KM)", "Doors"])
car_sales_filled

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431.0,4.0
1,BMW,Blue,192714.0,5.0
2,Honda,White,84714.0,4.0
3,Toyota,White,154365.0,4.0
4,Nissan,Blue,181577.0,3.0
...,...,...,...,...
945,Toyota,Black,35820.0,4.0
946,missing,White,155144.0,3.0
947,Nissan,Blue,66604.0,4.0
948,Honda,White,215883.0,4.0


In [14]:
car_sales_filled.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

#### -> converting categorical columns into numerical columns

In [15]:
one_hot = OneHotEncoder()

In [16]:
# defining columns
categorical_features = ["Make", "Colour", "Doors"]

In [17]:
transformer = ColumnTransformer([("one_hot",  one_hot, categorical_features)],
                                remainder = "passthrough")

In [18]:
transformed_X = transformer.fit_transform(car_sales_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [19]:
pd.DataFrame(transformed_X)

,0
0,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
1,"(0, 0)\t1.0\n (0, 6)\t1.0\n (0, 13)\t1.0\n..."
2,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
3,"(0, 3)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
4,"(0, 2)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
...,...
945,"(0, 3)\t1.0\n (0, 5)\t1.0\n (0, 12)\t1.0\n..."
946,"(0, 4)\t1.0\n (0, 9)\t1.0\n (0, 11)\t1.0\n..."
947,"(0, 2)\t1.0\n (0, 6)\t1.0\n (0, 12)\t1.0\n..."
948,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."


#### -> fitting the model (cross validation)

In [20]:
np.random.seed(42)

In [21]:
model = RandomForestRegressor()
cross_val_score(model, transformed_X, y, cv=3)

array([0.15807138, 0.16866354, 0.30217153])

### -> evaluating the regression cv model

In [22]:
# 1. r-squared
cvs_r2 = cross_val_score(model, transformed_X, y, cv=3, scoring = None)
cvs_r2

array([0.15370228, 0.18040009, 0.30570186])

In [23]:
np.mean(cvs_r2)

0.2132680753705181

In [24]:
# 2. mean squared error
cvs_mse = cross_val_score(model, transformed_X, y, cv=10, scoring="neg_mean_squared_error")
cvs_mse

array([-56046476.10586293, -61277054.93903578, -58372599.51797462,
       -44344291.71877205, -57344969.08705394, -44396271.97017809,
       -50014430.40114077, -63015439.66347518, -73868508.69254376,
       -49649248.19280922])

In [25]:
np.mean(cvs_mse)

-55832929.028884634

In [26]:
# 3. mean absolute error
cvs_mae = cross_val_score(model, transformed_X, y, cv=10, scoring = "neg_mean_absolute_error")
cvs_mae

array([-6170.78915259, -6319.38705308, -6272.78004112, -5305.50696659,
       -6187.57369264, -5268.00329379, -5580.27136566, -6304.92629093,
       -6654.27707431, -5800.24140029])

In [27]:
np.mean(cvs_mae)

-5986.375633100377